In [1]:
BATCH_SIZE = 256
LEARNING_RATE = 1e-3
QUBITS = 166
NEURONS_FROM = 0
NEURONS_PREDICTED = 166
EPOCHS = 4
MODE = "Train"
DEVICE = "cpu"

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_algorithms.utils import algorithm_globals

import torch
from torch import cat, no_grad, manual_seed, Tensor, nn
from torch.utils.data import DataLoader as TorchDataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import (
    Module,
    Conv2d,
    Linear,
    Dropout2d,
    PoissonNLLLoss,
    CrossEntropyLoss,
    MaxPool2d,
    AvgPool2d,
    Flatten,
    Sequential,
    ReLU,
    ELU
)
import torch.nn.functional as F

from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit_machine_learning.connectors import TorchConnector

from qiskit_aer import AerSimulator
aersim = AerSimulator()

algorithm_globals.random_seed = 42



import csv
import logging
import os
import time

import matplotlib

from cadena_ploscb.data import Dataset, MonkeySubDataset

In [3]:
# device = torch.device("cpu")

device = torch.device(
    "cuda") if (torch.cuda.is_available() and DEVICE == "gpu") else torch.device("cpu")

print(device)

torch.cuda.set_per_process_memory_fraction(0.5)

# torch.cuda.set_per_process_memory_fraction(0.5, 0)
torch.cuda.empty_cache()

cpu


In [4]:
def lin_step(x, a, b):
    return torch.minimum(torch.tensor(b - a, dtype=torch.float32), 
      F.relu(x - torch.tensor(a, dtype=torch.float32))) / (b - a)

def tent(x, a, b):
    z = torch.tensor(0, dtype=torch.float32).to(device)
    d = torch.tensor(2 * (b - a), dtype=torch.float32).to(device)
    a = torch.tensor(a, dtype=torch.float32).to(device)
    return torch.min(torch.max(x - a, z), torch.max(a + d - x, z)) / (b - a)

#Function used to map the Model output to neuron firing rates
def output_nonlinearity(x, vmin=-3.0, vmax=6.0, num_bins=10):
    elu = torch.nn.functional.elu(x - 1.0).to(device) + 1.0
    _, neurons = x.shape

    k = int(num_bins / 2)
    num_bins = 2 * k
    bins = np.linspace(vmin, vmax, num_bins + 1, endpoint=True)
    segments = [tent(x, a, b) for a, b in zip(bins[:-2], bins[1:-1])] + [lin_step(x, bins[-2], bins[-1])]

    a = nn.Parameter(torch.zeros(neurons, num_bins, 1, 
      dtype=torch.float32)).to(device)
    a_exp = torch.exp(a)
    v = torch.cat([torch.reshape(s, (-1, neurons, 1)) for s in segments], dim=2).to(device).permute(1, 0, 2)
    multiplier = torch.matmul(v, a_exp).view(neurons, -1).t()
    return multiplier * elu


In [5]:
def inv_elu(x):
    """Inverse elu function."""
    y = x.copy()
    idx = y < 1.0
    y[idx] = np.log(y[idx]) + 1.0
    return y
    
data_dict = Dataset.get_clean_data()
dataset = MonkeySubDataset(data_dict, seed=1000, train_frac=0.8, subsample=2, crop=30)

# b_out is calculated as inverse ELU of responses mean
# Used as input for non-linear mapping the model output to neuron firing rates
_, responses, _ = dataset.train()
b_out = torch.tensor(inv_elu(responses.mean(axis=0))).to(device)
b_out = b_out[NEURONS_FROM:NEURONS_FROM+QUBITS]

Subtracting mean: 112.51203


In [6]:
# Define Hybrid torch NN module
import torch.nn.init as init

class Net(Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2d(1, 32, kernel_size=13,padding='valid').to(device)
        self.conv2 = Conv2d(32, 32, kernel_size=3,padding='same').to(device)
        self.conv3 = Conv2d(32, 32, kernel_size=3,padding='same').to(device)
        self.pool = AvgPool2d(kernel_size=5, stride=1, padding=2)
        self.fc1 = Linear(32*28*28, 32*QUBITS)  # 32 * QUBIT-dimensional output
        self.fc2 = Linear(32*QUBITS, QUBITS)  # QUBIT-dimensional output
        # self.qnn = TorchConnector(qnn)  # Apply torch connector
        # self.conv4 = Conv2d(1, QUBITS, (QUBITS, 1)).to(device)
        init.trunc_normal_(self.conv1.weight, mean=0.0, std=0.01)
        # W_spatial = torch.randn(QUBITS, 1).to(device) * 0.01
        # W_spatial_flat = W_spatial.view(QUBITS, 1, 1, 1)
        
        # self.conv4.weight.data = self.conv4.weight.data + W_spatial_flat
        self.W_features = nn.Parameter(torch.randn(1, NEURONS_PREDICTED).to(device) * 0.01)
        self.elu = ELU()
    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = self.elu(self.conv1(x))
        x = self.elu(self.conv2(x))
        x = self.elu(self.conv3(x))
        print("X after conv 3", x.shape)

        # x = self.pool(x)
        # print("X after pool", x.shape)
        # x = self.fc1(torch.flatten(x, start_dim=1))
        # x = self.fc2(x)
        # x = self.elu(self.qnn(x))
        sz = x.shape
        px_x_conv = sz[2]
        px_y_conv = sz[3]
        px_conv = px_x_conv * px_y_conv
        conv_flat = x.reshape(-1, 1, 32, px_conv).to(device)
        print("conv_flat", conv_flat.shape)

        W_spatial = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty(px_conv, NEURONS_PREDICTED), mean=0.0, std=0.01)).to(device)
        print("W_Spatial", W_spatial.shape)
        W_spatial_flat = W_spatial.view(px_conv, 1, 1, NEURONS_PREDICTED).to(device)
        print("W_spatial_flat", W_spatial_flat.shape)
        h_spatial = nn.functional.conv2d(conv_flat, W_spatial_flat, stride=1, padding=0).to(device)  # dot product
        print("W_spatial", W_spatial.shape, "W_spatial_flat", W_spatial_flat.shape, "h_spatial", h_spatial.shape, "self.W_features", self.W_features.shape)
        h_out = torch.sum(h_spatial * self.W_features, dim=(1, 2)).to(device)
        # conv_flat = x.reshape(-1, 1, px_conv, 1)
        # h_spatial = self.elu(self.conv4(conv_flat)) #batch x 1 x 32 x neurons
        # h_spatial = h_spatial.permute(0, 2, 3, 1)
        # h_out = torch.sum(torch.mul(h_spatial, self.W_features), dim=(1, 2))
        pred = output_nonlinearity(h_out + b_out, vmin=-3.0, vmax=6.0, num_bins=10)
        return pred

model = Net().to(device)

In [7]:
def log_uniform(low=0, high=1, base=10):
    """draw samples from a uniform distribution in logspace"""
    return np.power(base, np.random.uniform(low, high))
    

def smoothness_regularizer_2d(W, weight=1.0):
    # Define the Laplacian kernel
    lap = torch.tensor([[0.25, 0.5, 0.25], [0.5, -3.0, 0.5], [0.25, 0.5, 0.25]]).unsqueeze(0).unsqueeze(0)
    lap = lap.repeat(W.size(1), 1, 1, 1)  # Assuming W is in shape (out_channels, in_channels, H, W)
    lap = lap.to(W.device)  # Ensure the kernel is on the same device as W

    # Apply depthwise convolution using groups in Conv2d
    out_channels, in_channels, _, _ = W.size()
    W_lap = F.conv2d(W.permute(1, 0, 2, 3), lap, groups=in_channels, padding='same')

    # Calculate penalty
    penalty = torch.sum(W_lap ** 2, dim=[1, 2, 3]) / torch.sum(W ** 2, dim=[2, 3])
    penalty = weight * torch.sum(penalty)

    return penalty

def group_sparsity_regularizer_2d(W, weight=1.0):
    # Calculate the sum of squares of W, then sum over the 0th and 1st dimension (assuming W is in shape of [C, H, W] or [C, H, W, D])
    sum_of_squares = torch.sum(W ** 2, dim=[0, 1])
    
    # Calculate the square root of the sum of squares, then sum over all remaining dimensions
    sqrt_sum_of_squares = torch.sqrt(sum_of_squares)
    penalty = torch.sum(sqrt_sum_of_squares)
    
    # Multiply by the weight and return
    penalty = weight * penalty

    return penalty

def smoothness_regularizer_1d(w, weight=1.0, order=2):
    penalty = 0
    # Define the kernel for 1D convolution
    kernel = torch.tensor([-1.0, 1.0], dtype=torch.float32).reshape(1, 1, 2).to(w.device)

    for _ in range(order):
        # Apply 1D convolution
        w = F.conv1d(w, kernel, stride=1, padding='valid')
        # Calculate and accumulate penalty
        penalty += torch.sum(torch.mean(w ** 2, dim=1))
    
    # Apply weight to the penalty
    penalty = weight * penalty

    return penalty

In [8]:
# PLOT = True
# BATCH_SIZE = 256
# if(PLOT):
#   rows = 10
#   fig, axes = plt.subplots(nrows=rows, ncols=rows, sharex=True, figsize=(20, 20))
#   img_batch, res_batch, real_batch = dataset.minibatch(BATCH_SIZE)
#   print("img_batch", img_batch.shape)
#   images = torch.tensor(img_batch).permute(0, 3, 1, 2)
#   print("images", images.shape)
#   print("squeezed", images[0].squeeze().shape)
#   for p in range(rows * rows):
#     axes[p//rows,p%rows].imshow(images[p].squeeze(), cmap="gray")
#     axes[p//rows,p%rows].set_xticks([])
#     axes[p//rows,p%rows].set_yticks([])

#   fig, axes = plt.subplots(nrows=rows, ncols=rows, sharex=True, figsize=(30, 30))
#   for p in range(rows * rows):
#     axes[p//rows,p%rows].imshow(img_batch[p].squeeze(), cmap="gray")
#     axes[p//rows,p%rows].set_xticks([])
#     axes[p//rows,p%rows].set_yticks([])


In [9]:
from scipy import stats
from torchmetrics.regression import PearsonCorrCoef

def evaluate_corr_vals(pred, res, real_res):
  corrs = []
  for i in range(NUM_NEURONS):
    # keep only entries corresponding to real_res
    r = res[:, i]
    p = pred[:, i]
    b = real_res[:, i].astype(bool)
    r = np.compress(b, r)
    p = np.compress(b, p)
    corr = stats.pearsonr(r, p)[0]
    if np.isnan(corr):
        print("INFO: corr for neuron " + str(i) + " is nan - replaced by 0")
        corr = 0
    corrs.append(corr)

  return corrs

def evaluate_avg_corr_val(pred, res, real_res):
  """Prediction correlation averaged across neurons on validation set."""
  avg_corr = np.mean(evaluate_corr_vals(pred, res, real_res))
  return avg_corr

def evaluate_pearson_correl(pred, res, real_res):
  pearson = PearsonCorrCoef(num_outputs=pred.shape[1]).to(device)
  eff_res = res * real_res
  return torch.mean(pearson(pred, eff_res))

class CustomLoss(nn.Module):
  def __init__(self):
    super(CustomLoss, self).__init__()

  def forward(self, prediction, responses_pair):
    response, real_responses = responses_pair
    # Calculate the loss
    loss = torch.mean(
        torch.sum((prediction - response * torch.log(prediction + 1e-9)) * real_responses, dim=0)
        / torch.sum(real_responses, dim=0)
    )
    return loss

In [10]:
loss_list = []  # Store loss history

def train():
  data_dict = Dataset.get_clean_data()
  dataset = MonkeySubDataset(data_dict, seed=1000, train_frac=0.8, subsample=2, crop=30)

  smooth_weights = torch.tensor([log_uniform(low=-9, high=-3.5), 0, 0]).to(device)
  sparse_weights = [0.0, 0.00025, 0.00025]
  readout_sparse_weight = log_uniform(low=-5.5, high=-1)
  trainSize = dataset.train()
  print(trainSize[0].shape[0])

  optimizer = optim.Adam(model.parameters(), lr=0.001)
  loss_func = nn.CrossEntropyLoss()
  num_neurons= 166 #int(QUBITS/2)

  # Start training
  epochs = EPOCHS  # Set number of epochs
  model.train()  # Set model to training mode

  log_interval = 10

  for epoch in range(epochs):
    total_loss = []

    not_improved = 0
    num_lr_updates = 0
    dataset.next_epoch()

    for p in range(500):
      if(BATCH_SIZE*(p+1) > trainSize[0].shape[0]):
        # for name, param in model.named_parameters():
        #   if param.requires_grad and "weight" in name:
        #     print(name, param.data.shape, param.data[0, 0, 0, :3].data)    
        imgs_val, res_val, real_resp_val = dataset.val()
        data_val = torch.Tensor(imgs_val).to(device)
        target_val = torch.Tensor(res_val).to(device)
        real_target_val = torch.Tensor(real_resp_val).to(device)
        target_val = target_val[:,NEURONS_FROM:NEURONS_FROM+QUBITS]
        real_target_val = real_target_val[:,NEURONS_FROM:NEURONS_FROM+QUBITS]
        output_val = model(data_val)
        loss = loss_func(output_val, target_val)
        # pearson = evaluate_pearson_correl(output_val, target_val, real_target_val)
        print(epoch, "step ", p, "Loss ", loss.item())
        break
      imgs_batch, res_batch, real_batch = dataset.minibatch(BATCH_SIZE)
      data = torch.Tensor(imgs_batch).to(device)
      target = torch.Tensor(res_batch).to(device)
      target = target[:,NEURONS_FROM:NEURONS_FROM+QUBITS]
      real_target = torch.Tensor(real_batch).to(device)
      output = model(data)
      # print(output[0].data, target[0].data)
      # print(p, "output", output.data)
      loss = loss_func(output, target)  # Calculate loss
      penalty1 = 0.0
      penalty2 = 0.0
      penalty3 = 0.0
      for name, param in model.named_parameters():
        if 'conv1.weight' in name:  # Assuming you want to regularize weights
          penalty1 = smoothness_regularizer_2d(torch.permute(param, (2, 3, 0, 1)), smooth_weights[0]) + group_sparsity_regularizer_2d(param, sparse_weights[0])
        # elif 'conv2.weight' in name:  # Assuming you want to regularize weights
        #   penalty2 = smoothness_regularizer_2d(torch.permute(param, (2, 3, 0, 1)), smooth_weights[1]) + group_sparsity_regularizer_2d(param, sparse_weights[1])
        # elif 'conv3.weight' in name:  # Assuming you want to regularize weights
        #   penalty3 = smoothness_regularizer_2d(torch.permute(param, (2, 3, 0, 1)), smooth_weights[2]) + group_sparsity_regularizer_2d(param, sparse_weights[2])
      # for name, param in model.named_parameters():
      #   if param.requires_grad and name == 'qnn.weight':
      #     print(name, param.data)      
      loss += penalty1 + penalty2 + penalty3
      print(epoch, p, "Loss ", loss.item(), "MSE", torch.mean((target - output) ** 2).item() # output.T[:, :5].data, target.T[:, :5].data, 
      )
      pearson = evaluate_pearson_correl(output, target, real_target[:,NEURONS_FROM:NEURONS_FROM+NEURONS_PREDICTED])
      optimizer.zero_grad()  # Initialize gradient set_to_none=True
      loss.backward()  # Backward pass
      optimizer.step()  # Optimize weights
      total_loss.append(loss.item())  # Store loss
      loss_list.append(sum(total_loss) / len(total_loss))
    # print("Training [{:.0f}%]\tLoss: {:.4f}".format(100.0 * (epoch + 1) / epochs, loss_list[-1]))

if(MODE == "Train"):
  train()

Subtracting mean: 112.51203
18560
X after conv 3 torch.Size([256, 32, 28, 28])
conv_flat torch.Size([256, 1, 32, 784])
W_Spatial torch.Size([784, 166])
W_spatial_flat torch.Size([784, 1, 1, 166])


: 

In [ ]:

if(MODE == "Train"):
  plt.plot(loss_list)
  plt.title("Hybrid QNN Training Convergence")
  plt.xlabel("Training Iterations")
  plt.ylabel("Poisson NLL Loss")
  plt.show()

In [ ]:
# Evaluate, calculate FEV
images_test, responses_test, real_responses_test = dataset.test()
print("images_test", images_test.shape, "responses_test", responses_test.shape, "real_responses_test", real_responses_test.shape)
# print("real_respones", real_responses_test[0, 0:10, 0:4])
# print("respones", responses_test[0, 0:10, 0:4])
total_count = images_test.shape[0]
print("total_count", total_count)
iterations = total_count//BATCH_SIZE +1
nrep, nim, tneu = responses_test.shape
nneu = QUBITS
fneu = NEURONS_FROM

predictions = torch.zeros(1, 166)

for i in range(iterations):
  data = torch.Tensor(images_test[i*BATCH_SIZE:(i+1)*BATCH_SIZE]).to(device)
  predictions_test = model(data)
  if i == 0:
    predictions = predictions_test
  else:
    predictions = torch.cat((predictions, predictions_test))
# print("predictions", predictions[0:5, :])


In [ ]:
def nanarray(real_resps, resps):
  """Inserts nan at every position in resps where corresponding positions in real_resps are 0.
  Hence, leading to an array of resps where non-real responses have value nan.
  """

  return np.where(real_resps, resps, np.nan)

  #Calculate Fraction of Explainable Variance
def calculateFEV3(predictions_test, responses_test, real_responses_test, 
  nrep, nim, tneu, nneu, fneu):
  predictions_test = torch.tile(predictions_test, (4, 1)).detach().cpu().numpy()
  resps_test_nan = dataset.nanarray(real_responses_test, responses_test)
  resps_test = resps_test_nan.reshape([nrep * nim, tneu])
  sresps_test = resps_test[:,fneu:fneu+nneu]
  print("sresps_test", sresps_test.shape, "predictions_test", predictions_test.shape)
  MSE = np.nanmean((predictions_test - sresps_test) ** 2, axis=0)
  print("MSE", MSE)
  obs_var_avg, total_variance, explainable_var = [], [], []
  for n in range(tneu):
    if(n < NEURONS_FROM or n >= NEURONS_FROM + QUBITS):
      continue
    print("processing for neuron", n)
    rep = dataset.repetitions[n]
    resp_ = resps_test_nan[:rep, :, n]
    print("resp_", resp_.shape, resp_[:, 0:4])
    v = np.nanvar(resp_, axis=0, ddof=1)
    print("v", v.shape, v[0:4])
    obs_var = np.nanmean(v, axis=0)
    print("obs_var", obs_var)
    obs_var_avg.append(obs_var)
    tot_var = np.nanvar(resp_, axis=(0, 1), ddof=1)
    print("tot_var", tot_var)
    total_variance.append(tot_var)
    explainable_var.append(tot_var - obs_var)
  total_variance = np.array(total_variance)
  explainable_var = np.array(explainable_var)
  var_explained = total_variance - MSE
  print(
    "indices", np.nonzero(MSE < total_variance),
    "MSE", MSE[MSE < total_variance], 
    "total_variance", total_variance[MSE < total_variance], 
    "var_explained", var_explained[MSE < total_variance], 
    "explainable_var", explainable_var[MSE < total_variance], 
    "fev%", (var_explained * 100 / explainable_var)[MSE < total_variance])
  return var_explained / explainable_var

In [ ]:
fev = calculateFEV3(predictions, responses_test, real_responses_test, nrep, nim, tneu, nneu, fneu)
print("FEV", fev[fev > 0]*100, fev[fev > 0].mean() * 100, "%")